# 002 - CTE

```SQL
DROP TABLE IF EXISTS registros_usuarios;

CREATE TABLE registros_usuarios (
    usuario_id VARCHAR(50),
    nombre_completo VARCHAR(100),
    correo_electronico VARCHAR(100),
    ciudad_residencia VARCHAR(50),
    puntos_lealtad INTEGER
);

INSERT INTO registros_usuarios VALUES
    ('U001', '  JUAN PEREZ', 'juan.perez@gmail.com', 'MÉXICO', 150),
    ('U002', 'marta gomez  ', 'Marta.Gomez@Gmail.com', 'mexico', 200),
    ('U002', 'marta gomez  ', 'Marta.Gomez@Gmail.com', 'mexico', 200), -- Duplicado
    ('U003', 'Carlos Slim', 'carlos@slim.com', 'CDMX', -500),         -- Puntos negativos (Error)
    ('U004', 'Ana Lopez', NULL, 'guadalajara', 10),                 -- Correo Nulo
    ('U005', '  Luisa Ortega', 'luisa@empresa.com', 'mExIcO', 10000); -- Outlier y texto raro

SELECT * FROM registros_usuarios;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_register = pd.read_csv('DATA/002/registros_usuarios_2.csv')

df_register.head()

,usuario_id,nombre_completo,correo_electronico,ciudad_residencia,puntos_lealtad
0,U001,JUAN PEREZ,juan.perez@gmail.com,MÉXICO,150
1,U002,marta gomez,Marta.Gomez@Gmail.com,mexico,200
2,U002,marta gomez,Marta.Gomez@Gmail.com,mexico,200
3,U003,Carlos Slim,carlos@slim.com,CDMX,-500
4,U004,Ana Lopez,NaN,guadalajara,10


# Ejercicio 1: "El Filtro de Texto e Identidad"

**Objetivo: Crear una CTE llamada usuarios_estandarizados que resuelva:**

- Quitar los espacios laterales del nombre.
- Poner la ciudad siempre en MAYÚSCULAS.
- Eliminar los duplicados exactos.


In [4]:
df_register['nombre_completo'] = df_register['nombre_completo'].str.strip()
df_register['ciudad_residencia'] = df_register['ciudad_residencia'].str.upper()

df_register = df_register.drop_duplicates()

respuesta1 =  df_register[['nombre_completo','ciudad_residencia']]

respuesta1

,nombre_completo,ciudad_residencia
0,JUAN PEREZ,MÉXICO
1,marta gomez,MEXICO
3,Carlos Slim,CDMX
4,Ana Lopez,GUADALAJARA
5,Luisa Ortega,MEXICO


```SQL
WITH eliminate_duplicate AS(
    SELECT DISTINCT * FROM registros_usuarios
    ),
usuarios_estandarizados AS (
    SELECT
        TRIM(nombre_completo) AS trim_nombre_completo,
        UPPER(ciudad_residencia) AS upper_ciudad
    FROM eliminate_duplicate
)
SELECT
    trim_nombre_completo,
    upper_ciudad
FROM usuarios_estandarizados;
```

# Ejercicio 2: "Manejo de Nulos y Valores lógicos" (Error 1 y 8)

**Objetivo: A partir de la tabla anterior, crea una CTE que:**

- Si el correo es NULL, cámbialo por 'SIN CORREO'.
- Si los puntos son negativos, conviértelos a 0.

In [ ]:
df_2 = df_register.copy()

df_2['correo_electronico'] = df_2['correo_electronico'].fillna('SIN CORREO')

df_2['puntos_lealtad'] = df_2['puntos_lealtad'].clip(lower=0)

df_2

,usuario_id,nombre_completo,correo_electronico,ciudad_residencia,puntos_lealtad
0,U001,JUAN PEREZ,juan.perez@gmail.com,MÉXICO,150
1,U002,marta gomez,Marta.Gomez@Gmail.com,MEXICO,200
3,U003,Carlos Slim,carlos@slim.com,CDMX,0
4,U004,Ana Lopez,SIN CORREO,GUADALAJARA,10
5,U005,Luisa Ortega,luisa@empresa.com,MEXICO,10000


```SQL
SELECT
    COALESCE(correo_electronico,'SIN CORREO') as email,
    (CASE WHEN puntos_lealtad < 0 THEN 0 ELSE puntos_lealtad END) as puntos
FROM registros_usuarios;
```

# Ejercicio 3: "El Combo de Limpieza" (Encadenando CTEs)

**Este es el ejercicio de nivel Snowflake. Vamos a encadenar dos CTEs: la primera limpia el texto y la segunda filtra los errores de negocio.**

Intenta escribir esta consulta:

- CTE 1 (datos_base): Limpia nombres (TRIM) y estandariza correos (LOWER).
- CTE 2 (datos_filtrados): Toma datos_base y filtra a los usuarios que viven en 'MÉXICO' (ya estandarizado) y que tienen puntos menores a 5000 (para quitar el outlier de Luisa).

In [31]:
df_3 = df_register.copy()
df_3['ciudad_residencia'] = df_3['ciudad_residencia'].str.upper().str.strip()
df_3['correo_electronico'] = df_3['correo_electronico'].str.lower().str.strip()

respuesta3 = df_3[
    (df_3['ciudad_residencia'] == 'MÉXICO') &
    (df_3['puntos_lealtad'] < 5000)
]

respuesta3 = respuesta3[['ciudad_residencia','puntos_lealtad']]

respuesta3

,ciudad_residencia,puntos_lealtad
0,MÉXICO,150


```SQL
WITH datos_base AS (
    SELECT
        UPPER(TRIM(ciudad_residencia)) AS upper_ciudad,
        LOWER(TRIM(correo_electronico)) AS lower_trim_email,
        puntos_lealtad
    FROM registros_usuarios
), datos_filtrados AS(
    SELECT
        upper_ciudad,
        puntos_lealtad
    FROM datos_base
    WHERE upper_ciudad = 'MÉXICO' AND puntos_lealtad < 5000
)
SELECT
    *
FROM datos_filtrados;
```